In [221]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xmltodict
import pprint
import re
import time
from gensim.parsing.preprocessing import remove_stopwords
from nltk.stem import *
%matplotlib inline
plt.style.use('ggplot')

In [222]:
with open('collections/trec.sample.xml', 'r', encoding='utf-8') as f:
    xml_file = f.read()

In [223]:
# load total text in the dict_file
dict_file = xmltodict.parse(xml_file)
dict_file = dict_file['document']['DOC']
# pprint.pprint(dict_file[999]) # index not equal to DOCNO

In [224]:
# dic['ID'] = HEADLINE + TEXT



dic = {}
for file in dict_file:
    new_content = file['HEADLINE'] + ' ' + file['TEXT'] # for trec.sample.xml
    # new_content = file['Text'] # for sample.xml
    punct = re.compile("[^\w\s]") # regax of token
    new_content = re.sub(punct, "", new_content) # tokenisation
    # new_content = new_content.lower() # make all text into lower case
    new_content = remove_stopwords(new_content) # use gensim to remove sw
    new_content = re.split('\s+', new_content) # convert string to list for stemming; or can use string.split()
    stemmer = SnowballStemmer("english")
    new_content = [stemmer.stem(word) if not word == stemmer.stem(word) else word for word in new_content] # stemming(include lower case)
    new_content = ' '.join(new_content) # convert list to string for writing into file
    dic['{}'.format(file['DOCNO'])] = new_content

In [225]:
# find all uniq terms
terms = {}
for docno in dic:
    for term in dic[docno].split():
        if hash(term) not in terms:
                terms[hash(term)] = term
len(terms)

19989

In [226]:
# positional index implementation
collection_dic = {}
for term_hash in terms:
    doc_dic = {}
    for docno in dic:
        # cannot use string.find() --> string index: char, list index: word(str)
        pos_list = [index for index, word in enumerate(dic[docno].split()) if terms[term_hash] == word]
        if len(pos_list):
            doc_dic[docno] = pos_list
    collection_dic[terms[term_hash]] = doc_dic

In [227]:
import pickle
import json
# write into txt and binary file
with open("collections/trec.english_stemmer.txt", 'w') as f1:
    f1.write(json.dumps(collection_dic))
with open("collections/trec.english_stemmer.dat", 'wb') as f2:
    pickle.dump(collection_dic, f2)